# Числовой ребус

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/60eeaf958fa73a6a989f00725cf7d4c3f516e929" width="200">

**Крипторитм** - математическая головоломка, пример арифметического действия, в котором все или некоторые цифры заменены буквами, звёздочками или другими символами.  
Задание состоит в том, чтобы восстановить исходную запись примера [wiki](https://ru.wikipedia.org/wiki/Числовой_ребус).

In [ ]:
from itertools import permutations

def solve_puzzle(X, Y, Z):
    '''Solve cryptarithms X + Y = Z

    >>> solution = solve_puzzle('АА', 'ББ', 'ВВ')
    >>> solution == {'А': '1', 'Б': '2', 'В': '3'}
    True
    '''
    digits = '0123456789'
    chars = sorted(set(X + Y + Z))
    for shuffle in permutations(digits, len(chars)):
        solution = {c: d for c, d in zip(chars, shuffle)}
        convert = lambda s: int(''.join(solution[c] for c in s))
        if convert(X) + convert(Y) == convert(Z):
            return solution

In [ ]:
solution = solve_puzzle('SEND', 'MORE', 'MONEY')
print(solution)

In [ ]:
puzzle = '''
  SEND
+ MORE
------
 MONEY
'''  
print(''.join([solution.get(c, c) for c in puzzle]))

In [ ]:
def parsing(request):
    '''Split the request into parts
    
    >>> parsing('AB + CDE = FG')
    ('AB', 'CDE', 'FG')
    '''
    summands, Z = request.split('=')
    X, Y = summands.split('+')
    return X.strip(), Y.strip(), Z.strip()

In [ ]:
parsing('SEND + MORE=     MONEY')

# Телеграм Бот

<img src="https://core.telegram.org/file/811140327/1/zlN4goPTupk/9ff2f2f01c4bd1b013" width="200">

#### План действий   
* пишем в телеграмме https://t.me/BotFather
* придумываем имя боту. У моего - [`CryptarithmsBezZabot`](https://t.me/CryptarithmsBezZabot)
* получаем токен от BotFather\`а (он потребуется для подключения)
* следуя [туториалу](https://www.gitbook.com/book/groosha/telegram-bot-lessons/details), реализуем возможность решать  головоломку
* [официальная страница телеграма про бота](https://core.telegram.org/bots)

Устанавливаем API для обращения с телеграмм ботом [github](https://github.com/eternnoir/pyTelegramBotAPI#getting-started).

In [ ]:
import telebot # !pip install pyTelegramBotAPI

Используя magic, записываем выданный токен в файл для того, чтобы не светить его в коде.

In [ ]:
# %%writefile config.py
# tocken = '<сюда вставить токен>'

Импортируем наш токен из записанного файла

In [ ]:
from config import tocken

Создаем бота

In [ ]:
bot = telebot.TeleBot(tocken) # Создаем бота

Функция обработки одного сообщения

In [ ]:
@bot.message_handler(content_types=["text"]) # Декоратор - изменяет поведение функции
def repet_to(message):        # Название функции не играет никакой роли
    chat_id = message.chat.id # message содержит все необходимые данные
    request = message.text
    try:                           # Если возникнет ошибка, то мы ее обработаем
        X, Y, Z = parsing(request) # Скорее всего ошибка будет при попытке разбора сообщения
        bot.send_message(chat_id, '🤔 посмотрим ...') # Отправлем сообщение (нужно указать чат!)
        solution = solve_puzzle(X, Y, Z)
        if solution:
            answer = ''.join([solution.get(c, c) for c in request])
            bot.send_message(chat_id, answer)
        else:
            bot.send_message(chat_id, '😒 решений нет')
    except Exception as e: # Ловим исключение
        bot.send_message(chat_id, '😳 не понял?') # Отправляем сообщение

Запускаем бесконечный цикл на прослушивание сообщений.  
Чтобы остановиться, нужно бросить **`KeyboardInterrupt`**.

In [ ]:
bot.polling(none_stop=True)

Бот отвечает на сообщения, но чтобы он работал, нужно оставить сессию Питона на ноутбуке 😒.

### https://www.pythonanywhere.com


Есть бесплатные сервера, на которых можно запускать нашего бота.  
Я использую - **`pythonanywhere`** (можно покрутить скрипт в течении 24 часов).

### Сначала регистрируемся
Все стандартно: почта и придумать пароль.

### Установка окружения
Находима bash консоль. Нам нужно установить API телеграмма. 
Выполняем:
```bash
pip3.6 install --user pyTelegramBotAPI
```
### Копирование кода
Теперь через веб интерфейс создадим два питоновских файлика.


#### `config.py`
```python
tocken = <сюда вставить токен>
```

#### `cryptarithms.py`
```python
#!/usr/bin/python3.6
# -*- coding: UTF-8 -*-
import telebot
from itertools import permutations

from config import tocken


def solve_puzzle(X, Y, Z):
    '''Solve cryptarithms X + Y = Z

    >>> solution = solve_puzzle('AA', 'BB', 'CC')
    >>> solution == {'A': '1', 'B': '2', 'C': '3'}
    True
    '''
    digits = '0123456789'
    chars = sorted(set(X + Y + Z))
    for shuffle in permutations(digits, len(chars)):
        solution = {c: d for c, d in zip(chars, shuffle)}
        convert = lambda s: int(''.join(solution[c] for c in s))
        if convert(X) + convert(Y) == convert(Z):
            return solution


def parsing(request):
    '''Split the request into parts

    >>> parsing('AB + CDE = FG')
    ('AB', 'CDE', 'FG')
    '''
    summands, Z = request.split('=')
    X, Y = summands.split('+')
    return X.strip(), Y.strip(), Z.strip()


bot = telebot.TeleBot(tocken)


@bot.message_handler(content_types=["text"]) # Декоратор - изменяет поведение определяемой функции
def repet_to(message):        # Название функции не играет никакой роли
    chat_id = message.chat.id # message содержит все необходимые данные
    request = message.text
    print(message.text)
    try:                           # Если возникнет ошибка, то мы ее обработаем
        X, Y, Z = parsing(request) # Скорее всего ошибка будет при попытке разбора сообщения
        bot.send_message(chat_id, '🤔 посмотрим ...') # Отправлем сообщение (нужно указать чат!)
        solution = solve_puzzle(X, Y, Z)
        if solution:
            answer = ''.join([solution.get(c, c) for c in request])
            bot.send_message(chat_id, answer)
        else:
            bot.send_message(chat_id, '😒 решений нет')
    except Exception: # Ловим исключение
        bot.send_message(chat_id, '😳 не понял?') # Отправляем сообщение


if __name__ == '__main__':
    bot.polling(none_stop=True)

  ```

### Запуск выполнения

Опять открываем консоль и выполняем:
```bash
python3.6 cryptarithms.py
```

### Итог

Бот должен теперь крутиться на сервере и отвечать на сообщения.

# Данмоеше знааиде

По рзелульаттам илссеовадний одонго анлигйсокго унвиертисета, не иеемт 
занчнеия, в кокам пряокде рсапожолены бкувы в солве. Галвоне, чотбы 
преавя и пслоендяя бквуы блыи на мсете. Осатьлыне бкувы мгоут селдовтаь 
в плоонм бсепордяке, все-рвано ткест чтаитсея без побрелм. 
Пичрионй эгото ялвятеся то, что мы не чиатем кдаужю бкуву по 
отдльенотси, а все солво цликеом. 

Зиадане зюаачслктея в том, чбтоы сдеалть тламгреем бтоа, котрыой будет пашеертевмиь бкувы в слаовх из сеоноищбя. Првеая и поясндлея бвкуы длонжы оастьтся на метсе.

In [ ]:
from random import shuffle

def mix_letters_in(word):
    '''Permutation of letters
    
    >>> mix_letters_in('Мы')
    'Мы'
    >>> mix_letters_in('англичане')
    'ачанглие'
    
    '''
    pass

def british_mixing(text):
    '''Everyday i'm shuffling
    
    >>> british_mixing('Британские ученые доказали, что ...')
    'Брсиитнаке унычее дзаколаи, что ...'
    '''
    pass